### Importando os dados da camada Bronze

- Criando o DF com Pandas
- Mostrando os primeiros registros

In [0]:
import pandas as pd 

#Importando os caminhos para os dataset na bronze
url_dataset_training = '/Volumes/workspace/bronze/car_insurance_training/car_insurance_premium_dataset.csv'
url_dataset_test = '/Volumes/workspace/bronze/car_insurance_training/car_insurance_premium_dataset_TEST.csv'

#Criando o DF
df_car_insurance_tr = pd.read_csv(url_dataset_training)
df_car_insurance_test = pd.read_csv(url_dataset_test)

display(df_car_insurance_tr.head())
display(df_car_insurance_test.head())

## Tarefas a serem feitas
###1) Aplicar lowercase em todas as colunas;

In [0]:
#Função para lowercase em DataFrames
def columns_lower(df):
    #Transforma todas colunas em minusculo
    df.columns = df.columns.str.lower()
    return df.head()

columns_lower(df_car_insurance_tr)
columns_lower(df_car_insurance_test)

### 2) Excluir caracteres especiais dos nomes das colunas, incluindo o espaço em branco entre nome de colunas;


In [0]:
#Função para limpar colunas
def column_clear(df):
    #Tira os espaços e insere underline
    df.columns = df.columns.str.replace(' ', '_')
    #Tira os caracteres especiais
    df.columns = df.columns.str.replace(r'[^0-9a-zA-Z_]+','', regex = True)
    return df.head()

column_clear(df_car_insurance_tr)
column_clear(df_car_insurance_test)

### 3) Tratamento dos outliers usando IQR e ZScore;
##### A) Usando IQR

In [0]:
#Função para retornar cada outlier das colunas
def is_outlier(df):

  outliers = pd.DataFrame()
  for col in df.columns:

    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3-q1

    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr

    #Filtra as colunas onde valores são outilers
    outliers = pd.concat([outliers, df[(df[col] < lim_inf) | (df[col] > lim_sup)]])


  return outliers.drop_duplicates()

is_outlier(df_car_insurance_tr)

### Nenhum Outlier identificado em nenhuma coluna

- Vou criar um dataframe com outliers para verificar se a função não consegue capturar ou se realmente não existem no dataset

Identificando quais são os limites de cada coluna para "fabricar" outliers

In [0]:
#Exibindo os limites de cada coluna 
for col in df_car_insurance_tr.columns:

    q1 = df_car_insurance_tr[col].quantile(0.25)
    q3 = df_car_insurance_tr[col].quantile(0.75)
    iqr = q3-q1
    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr

    print(col)
    print(lim_inf)
    print(lim_sup)

- Criando um DataFrame com registros que são Outliers.
- Mesclando com nosso dataset em um Dataframe de teste

In [0]:
#Todas as colunas são outliers baseados nos limites encontrados
df = pd.DataFrame({
    'driver_age': [88],
    'driver_experience': [49],
    'previous_accidents': [9],
    'annual_mileage_x1000_km': [35],
    'car_manufacturing_year': [1971],
    'car_age': [54],
    'insurance_premium_': [512],

})

#Adicionando os outliers em um dataframe de teste
df_out = pd.concat([df_car_insurance_tr, df])
df_out.tail(3)


Testando a função usando o **DataFrame com Outliers**

In [0]:
import numpy  as np

#Função para retornar cada outlier das colunas
def is_outlier(df):

  outliers = pd.DataFrame()
  for col in df.columns:

    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3-q1

    lim_inf = q1 - 1.5 * iqr
    lim_sup = q3 + 1.5 * iqr

    #Filtra as colunas onde valores são outilers
    outliers = pd.concat([outliers, df[(df[col] < lim_inf) | (df[col] > lim_sup)]])


  return outliers.drop_duplicates()

#DF de teste onde a ultima linha são outliers
is_outlier(df_out)

#### Outliers "fabricados" foram encontrados pela função.
#### Função funciona, então **não existem outliers no DF original**.

##### B) Usando ZScore

- Normalizando as colunas

In [0]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


#Função que normaliza todas as colunas
def normal_columns(df):
  
  columns = df.columns

  for column in columns:
    df[column] = StandardScaler().fit_transform(df[[column]])
  
  return df.head()

#Criando a cópia do DF original para aplicar a normalização
df_insurance_ss_tr = df_car_insurance_tr.copy()
normal_columns(df_insurance_ss_tr)


Com as colunas normalizadas, vamos tentar filtrar onde Z > 3 ou Z < -3.

In [0]:
import numpy  as np

#Função para retornar cada outlier das colunas
def is_outlier_normal(df):

  outliers = pd.DataFrame()
  for col in df.columns:

    #Filtra as colunas onde valores são outilers
    outliers = pd.concat([outliers, df[(df[col] < -3) | (df[col] > 3)]])


  return outliers.drop_duplicates()

#DF de teste onde a ultima linha são outliers
is_outlier_normal(df_insurance_ss_tr)

##### Não foi encontrado nenhum valor < -3 ou > 3 — Sem outliers

- Mostrando as frequências acumuladas

In [0]:
#Mostra a frequência acumulada de todas as colunas
def freq_acc(df):

    columns = df.columns
    n = len(columns)
    fig, axes = plt.subplots(1, n, figsize=(16, 4))

    for i,column in enumerate(columns):
        axes[i].scatter(range(df.shape[0]), np.sort(df[column].values))
        axes[i].set_title(f"{column}")

    plt.tight_layout()
    plt.show()  

#Usa o DF com as colunas normalizadas
freq_acc(df_insurance_ss_tr)

#### Também não encontramos outliers usando o Zscore

- Vamos usar novamente o Dataframe com os "outliers fabricados" para testar

**Normalizando o DF com outliers**

In [0]:
#Normalizando o DF que inserimos os outliers para teste
df_out_ss = df_out.copy()
normal_columns(df_out_ss)

**Verificando se a função encontra outliers com o DF em que os inserimos**

In [0]:
#Função para retornar o DF com outliers
is_outlier_normal(df_out_ss)

**Valores encontrados, agora mostrando na frequência acumulada**

In [0]:
#Função de frequência acumulada
freq_acc(df_out_ss)

Pontos encontrados com o dataframe com Outliers.

##### Usando o dataframe com outliers, podemos ver que as funções funcionam. 
### Conclusão: Não existem Outliers no Dataframe original. 

### 4) Tratamento dos missing values;


In [0]:
#Não existem missing values
df_car_insurance_tr.isna().sum()
df_car_insurance_test.isna().sum()

### 5) Lidar com dados categóricos;

In [0]:
#Mostrando os DF
display(df_car_insurance_tr.head())
display(df_car_insurance_test.head())

### Não existem dados categóricos neste DF.

Não existem pois todas as colunas se tratam de dados númericos.

Uma das maneiras de determinar se Dados são categóricos é tentar calcular a média, por exemplo. Todas as colunas são possíveis disso neste DF.

- driver_age: Representa a idade do condutor - **Dados numéricos**
- driver_experience: Experiência em anos do condutor - **Dados numéricos**
- previous_accidents: Quantidade de acidentes prévios do motorista - **Dados numéricos**
- annual_mileage_x1000_km: Quilometragem por ano (/1000) - **Dados numéricos**
- car_manufacturing_year: Ano de fabricação - **Dados numéricos**
- car_age: Idade do carro - **Dados numéricos**
- insurance_premium_: Valor do seguro - **Dados numéricos**



#### Após realizar as etapas de limpeza dos dados, os resultados serão salvos na camada Silver

- Salvando na Silver

In [0]:
spark_df = spark.createDataFrame(df_car_insurance_tr)

spark_df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.t_car_insurance_tr")

In [0]:
df = spark.table('workspace.silver.t_car_insurance_tr').limit(5)
display(df)

### 6) Fazer EDA (Análise Expploratória de Dados);

In [0]:
df_car_insurance_tr.info()
#Colunas, quantidade de não-nulos e tipo dos dados

In [0]:
#Quantidade de linhas e colunas
df_car_insurance_tr.shape

In [0]:
#Estatísticas descritivas das colunas
df_car_insurance_tr.describe()

In [0]:
import seaborn as sns

#Gráfico de dispersão
sns.scatterplot(x='driver_experience', y='insurance_premium_', data=df_car_insurance_tr)

- Relação linear entre experiência do motorista e valor do seguro

In [0]:
# Cálculo das medidas de Skewness e Kurtosis para 'driver_age'
print(f"Skewness: {df_insurance_ss_tr['driver_age'].skew()}")
print(f"Kurtosis: {df_insurance_ss_tr['driver_age'].kurt()}")

- **Skewness**  ``0`` singifica que a variável 'age' é uma variável **simétrica**.
- **Kurtosis** ``-1`` significa que é uma distribuição platicúrtica (pico achatado e caudas leves - menos outliers)


In [0]:
sns.distplot(df_insurance_ss_tr['driver_age'])

### 7) Descreva os tipos de dados: float, int64, integer, ...


In [0]:
#acessando os tipos de dados de cada coluna do DF
df_car_insurance_tr.dtypes


- int64 / integer - números inteiros

- float64 / float - números com casas decimais

##### Salvando versão final na Gold

In [0]:
spark_df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.T_car_insurance_tr_gold")
